# Загрузка всех библиотек

In [4]:
!pip install aiogram
!pip install natasha

In [5]:
import asyncio
import os
from aiogram import Bot, Dispatcher, types
from aiogram.client.default import DefaultBotProperties
from aiogram.filters import CommandStart
from aiogram.enums import ParseMode
from aiogram.filters import Command
from aiogram.types import Message

from transformers import AutoTokenizer
import pickle
import nest_asyncio
import json
import torch
from typing import Dict, List, Any
from natasha import (
    Segmenter, MorphVocab, NewsEmbedding,
    NewsNERTagger, AddrExtractor, DatesExtractor,
    Doc
)
from transformers import pipeline


In [6]:
from google.colab import drive

drive.mount('/content/drive/')

Mounted at /content/drive/


# Загрузка категорий и модели

In [7]:
model_name = "cointegrated/rubert-tiny2"
tokenizer = AutoTokenizer.from_pretrained('/content/drive/MyDrive/Жалобы жкх/веса и бинарники/tiny2/ruber_riny2_tokenizer')

try:
    with open('/content/drive/MyDrive/Жалобы жкх/веса и бинарники/tiny2/tiny2_encoders.pkl', 'rb') as f:
        binarizers = pickle.load(f)
    print("Binarizers загружены!")
except FileNotFoundError:
    print("Файл binarizers не найден!")

Binarizers загружены!


Архитектура

In [8]:
import torch.nn as nn
from transformers import BertPreTrainedModel, BertModel

class SimpleMultiTaskBERT(BertPreTrainedModel):
    def __init__(self, config, num_categories, num_emotions, num_urgencies):
        super().__init__(config)
        self.bert = BertModel(config)
        self.dropout = nn.Dropout(0.8)


        self.category_dropout = nn.Dropout(0.9)

        self.classifier_category = nn.Linear(config.hidden_size, num_categories)
        self.classifier_emotion = nn.Linear(config.hidden_size, num_emotions)
        self.classifier_urgency = nn.Linear(config.hidden_size, num_urgencies)

    def forward(self, input_ids=None, attention_mask=None):
        outputs = self.bert(input_ids, attention_mask=attention_mask)
        pooled_output = outputs[1]


        category_output = self.category_dropout(pooled_output)
        emotion_output = self.dropout(pooled_output)
        urgency_output = self.dropout(pooled_output)

        category_logits = self.classifier_category(category_output)
        emotion_logits = self.classifier_emotion(emotion_output)
        urgency_logits = self.classifier_urgency(urgency_output)

        return {
            'category_logits': category_logits,
            'emotion_logits': emotion_logits,
            'urgency_logits': urgency_logits
        }

In [9]:
num_categories = len(binarizers['category']['classes'])  # 11
num_emotions = len(binarizers['emotion']['classes'])     # 6
num_urgencies = len(binarizers['urgency']['classes'])

In [10]:
from transformers import AutoConfig

model_name = 'cointegrated/rubert-tiny2'
config = AutoConfig.from_pretrained(model_name)

model = SimpleMultiTaskBERT.from_pretrained(
    model_name,
    config=config,
    num_categories=num_categories,
    num_emotions=num_emotions,
    num_urgencies=num_urgencies
)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/693 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/118M [00:00<?, ?B/s]

Some weights of SimpleMultiTaskBERT were not initialized from the model checkpoint at cointegrated/rubert-tiny2 and are newly initialized: ['classifier_category.bias', 'classifier_category.weight', 'classifier_emotion.bias', 'classifier_emotion.weight', 'classifier_urgency.bias', 'classifier_urgency.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
model.load_state_dict(torch.load('/content/drive/MyDrive/Жалобы жкх/веса и бинарники/tiny2/multitask_model_weights_tiny2.pth', map_location='cpu'))
model.eval()
print("Модель загружена")

Модель загружена


# Телергамм бот

In [12]:
from google.colab import userdata

In [13]:
TOKEN = userdata.get('TOKEN')

Загрузка json для формирования ответа

In [14]:
SPECIALISTS_PATH = '/content/drive/MyDrive/Жалобы жкх/rag/specialists.json'
DOCS_PATH = '/content/drive/MyDrive/Жалобы жкх/rag/jkh.json'

In [15]:
def load_json_data():
    """Load problems and specialists data from JSON files"""
    try:
        with open(DOCS_PATH, 'r', encoding='utf-8') as f:
            problems_data = json.load(f)

        with open(SPECIALISTS_PATH, 'r', encoding='utf-8') as f:
            specialists_data = json.load(f)

        return problems_data, specialists_data
    except Exception as e:
        print(f"Error loading JSON data: {e}")
        return {}, []

problems_data, specialists_data = load_json_data()

Анализ жалобы с помощью

json+NER(natasha)+Zero-Shot QA

In [16]:
class ImprovedComplaintAnalyzer:
    def __init__(self, problems_data, specialists_data):
        self.segmenter = Segmenter()
        self.morph_vocab = MorphVocab()
        emb = NewsEmbedding()
        self.ner_tagger = NewsNERTagger(emb)
        self.addr_extractor = AddrExtractor(self.morph_vocab)
        self.dates_extractor = DatesExtractor(self.morph_vocab)

        # Zero-Shot QA
        self.qa_pipeline = pipeline(
            "question-answering",
            model="microsoft/Multilingual-MiniLM-L12-H384",
            tokenizer="microsoft/Multilingual-MiniLM-L12-H384"
        )

        # RAG
        self.problems_data = problems_data
        self.specialists_data = specialists_data

        self.keyword_index = self.build_keyword_index()

    def build_keyword_index(self):
        """Build index of keywords for faster and better matching"""
        index = {}
        for problem_key, problem_info in self.problems_data.items():
            patterns = problem_info.get('patterns', [])
            for pattern in patterns:
                words = pattern.lower().split()
                for word in words:
                    if len(word) > 3:  # только значимые слова
                        if word not in index:
                            index[word] = []
                        index[word].append(problem_key)
        return index

    def analyze_complaint(self, text: str) -> Dict[str, Any]:
        """Анализирует жалобу на основе NER, QA and RAG"""

        ner_result = self.ner_analysis(text)
        qa_result = self.qa_analysis(text)
        rag_result = self.rag_search(text, ner_result)

        return {
            'ner_result': ner_result,
            'qa_result': qa_result,
            'rag_result': rag_result,
            'combined_analysis': self.combine_results(ner_result, qa_result, rag_result)
        }

    def ner_analysis(self, text: str) -> Dict[str, Any]:
        """Извлечение сущностей с помощью Natasha"""
        doc = Doc(text)
        doc.segment(self.segmenter)
        doc.tag_ner(self.ner_tagger)

        addresses = self.addr_extractor(text)
        dates = self.dates_extractor(text)

        return {
            'addresses': [str(addr.fact) for addr in addresses],
            'dates': [str(date.fact) for date in dates],
            'entities': [(span.text, span.type) for span in doc.spans],
            'problems': self.find_problems_ner(text)
        }

    def find_problems_ner(self, text: str) -> List[str]:
        """Поиск проблемы по ключевым словам"""
        problems = []
        text_lower = text.lower()

        keyword_groups = {
            'gas_leak': ['газ', 'утечка', 'запах газа', 'воняет газом', 'пахнет газом'],
            'water_leak': ['протекает', 'течет', 'протечка', 'подтопление', 'потоп', 'вода', 'кран', 'труба'],
            'heating': ['холодно', 'отопление', 'батареи', 'радиаторы', 'не греют', 'холодные'],
            'sewage': ['засор', 'канализация', 'забилось', 'не уходит', 'слив', 'унитаз'],
            'electricity': ['свет', 'электричество', 'пробки', 'выбило', 'автомат', 'электрик', 'горит', 'лампочка'],
            'garbage': ['мусор', 'контейнеры', 'вывоз', 'мусорка', 'отходы'],
            'elevator': ['лифт', 'застрял', 'подъемник', 'кабина'],
            'roof': ['крыша', 'кровля', 'протекает', 'потолок', 'чердак'],
            'noise': ['шум', 'соседи', 'громко', 'музыка', 'крики', 'орать']
        }

        for problem, words in keyword_groups.items():
            if any(word in text_lower for word in words):
                problems.append(problem)

        return problems

    def qa_analysis(self, text: str) -> Dict[str, Any]:
        """Извлечение информации с помощью Zero-Shot QA"""
        questions = [
            "Какая проблема описана в тексте?",
            "Где именно произошла проблема?",
            "Когда началась проблема?",
            "Что именно не работает или сломано?",
            "Какая срочность у проблемы?",
            "Какие контакты есть в тексте?"
        ]

        qa_results = {}
        for question in questions:
            try:
                result = self.qa_pipeline(question=question, context=text)
                if result['score'] > 0.2:  # Lowered threshold for better coverage
                    qa_results[question] = {
                        'answer': result['answer'],
                        'confidence': round(result['score'], 3)
                    }
            except Exception as e:
                continue

        return qa_results

    def rag_search(self, text: str, ner_result: Dict) -> Dict[str, Any]:
        """Поиск по json для ответа"""
        category = self.determine_category_improved(text, ner_result)
        relevant_problems = self.find_relevant_problems_improved(text, category)
        specialists = self.find_specialists(category, ner_result)

        return {
            'category': category,
            'relevant_problems': relevant_problems[:3],
            'specialists': specialists,
            'recommendations': self.generate_recommendations(relevant_problems, specialists, category)
        }

    def determine_category_improved(self, text: str, ner_result: Dict) -> str:
        """Определение категории"""
        text_lower = text.lower()

        for problem_key, problem_data in self.problems_data.items():
            patterns = problem_data.get('patterns', [])
            for pattern in patterns:
                if pattern.lower() in text_lower:
                    return problem_key

        category_scores = {}
        for problem_key, problem_data in self.problems_data.items():
            patterns = problem_data.get('patterns', [])
            score = 0
            for pattern in patterns:
                pattern_words = set(pattern.lower().split())
                text_words = set(text_lower.split())
                common_words = pattern_words.intersection(text_words)
                if common_words:
                    score += len(common_words)
            if score > 0:
                category_scores[problem_key] = score

        if category_scores:
            return max(category_scores.items(), key=lambda x: x[1])[0]

        ner_problems = ner_result.get('problems', [])
        if ner_problems:
            problem_mapping = {
                'electricity': 'electricity_supply_break',
                'heating': 'heating_system_break',
                'water_leak': 'water_supply_break',
                'sewage': 'sewage_system_break',
                'gas_leak': 'gas_leak_emergency',
                'elevator': 'elevator_breakdown',
                'roof': 'roof_leak',
                'noise': 'noise_violations',
                'garbage': 'garbage_chute_problems'
            }
            for ner_problem in ner_problems:
                if ner_problem in problem_mapping:
                    return problem_mapping[ner_problem]

        return 'other'

    def find_relevant_problems_improved(self, text: str, category: str) -> List[Dict]:
        """Находит проблему"""
        text_lower = text.lower()
        relevant_problems = []

        for problem_key, problem_data in self.problems_data.items():
            score = 0

            patterns = problem_data.get('patterns', [])
            for pattern in patterns:
                if pattern.lower() in text_lower:
                    score += 3
                else:
                    pattern_words = pattern.lower().split()
                    if any(word in text_lower for word in pattern_words if len(word) > 3):
                        score += 1

            if problem_key == category:
                score += 2

            text_words = set(text_lower.split())
            for word in text_words:
                if len(word) > 3 and word in self.keyword_index:
                    if problem_key in self.keyword_index[word]:
                        score += 1

            if score > 0:
                relevant_problems.append({
                    'key': problem_key,
                    'data': problem_data,
                    'relevance': min(score / 10.0, 1.0)
                })


        relevant_problems.sort(key=lambda x: x['relevance'], reverse=True)
        return relevant_problems

    def find_specialists(self, category: str, ner_result: Dict) -> List[Dict]:
        """Match specialists to problem"""
        required_specializations = self.determine_required_specializations(category)

        suitable_specialists = []
        for specialist in self.specialists_data:
            specializations = [s.lower() for s in specialist.get('specialization', [])]
            position = specialist.get('position', '').lower()

            matches = sum(1 for req in required_specializations
                       if any(req in spec for spec in specializations) or req in position)

            if matches > 0:
                suitable_specialists.append({
                    'specialist': specialist,
                    'relevance': matches,
                    'district': specialist.get('district', 'Не указан')
                })

        suitable_specialists.sort(key=lambda x: x['relevance'], reverse=True)
        return suitable_specialists[:5]

    def determine_required_specializations(self, category: str) -> List[str]:
        """Специалисты по проблемам"""
        mapping = {
            'gas_leak_emergency': ['аварийный', 'газ', 'сантехник'],
            'heating_system_break': ['отопление', 'сантехник', 'аварийный'],
            'electricity_supply_break': ['электрик', 'электричество', 'аварийный'],
            'sewage_system_break': ['сантехник', 'канализация', 'аварийный'],
            'water_supply_break': ['сантехник', 'вода', 'аварийный'],
            'elevator_breakdown': ['лифт', 'техник'],
            'roof_leak': ['кровля', 'кровельщик'],
            'noise_violations': ['диспетчер', 'координация']
        }
        return mapping.get(category, ['диспетчер'])

    def generate_recommendations(self, problems: List, specialists: List, category: str) -> Dict[str, Any]:
        """Генерацию рекоменлаций"""
        if problems:
            best_problem = problems[0]['data']
        else:
            fallback_recommendations = {
                'electricity_supply_break': {
                    'actions': ['Проверить автоматы в щитке', 'Вызвать электрика', 'Сообщить в диспетчерскую'],
                    'contacts': ['112', 'Аварийная служба УК'],
                    'deadlines': '2-4 часа'
                },
                'heating_system_break': {
                    'actions': ['Проверить соседей', 'Вызвать сантехника', 'Сообщить в аварийную службу'],
                    'contacts': ['8-800-100-50-50'],
                    'deadlines': '1-3 часа'
                },
                'water_supply_break': {
                    'actions': ['Перекрыть воду', 'Вызвать сантехника', 'Составить акт'],
                    'contacts': ['8-800-200-30-40'],
                    'deadlines': '4 часа'
                }
            }

            fallback = fallback_recommendations.get(category, {
                'actions': ['Обратитесь в диспетчерскую службу'],
                'contacts': ['8-800-100-00-00'],
                'deadlines': 'не указаны'
            })

            return {
                'urgency': 'средняя',
                'actions': fallback['actions'],
                'deadlines': fallback['deadlines'],
                'contacts': fallback['contacts'],
                'recommended_specialists': [s['specialist']['name'] for s in specialists[:3]],
                'legal_basis': 'Жилищный кодекс РФ'
            }

        return {
            'urgency': best_problem.get('urgency', 'средняя'),
            'actions': best_problem.get('actions', []),
            'deadlines': best_problem.get('deadline', 'не указаны'),
            'contacts': best_problem.get('contacts', []),
            'recommended_specialists': [s['specialist']['name'] for s in specialists[:3]],
            'legal_basis': best_problem.get('law', '')
        }

    def combine_results(self, ner_result: Dict, qa_result: Dict, rag_result: Dict) -> Dict[str, Any]:
        """Комбинация результатов со всех способов"""
        return {
            'extracted_data': {
                'addresses': ner_result['addresses'],
                'dates': ner_result['dates'],
                'problems': ner_result['problems'],
                'additional_info': qa_result
            },
            'recommendations': rag_result['recommendations'],
            'problem_category': rag_result['category'],
            'urgency_level': rag_result['recommendations']['urgency']
        }

    def generate_report(self, analysis_result: Dict) -> str:
        """Генерация отчета для пользователя"""
        data = analysis_result['combined_analysis']
        recommendations = data['recommendations']
        category_display = data['problem_category'].replace('_', ' ').title()

        report = [
            "🔍 **ПОЛНЫЙ АНАЛИЗ ЖАЛОБЫ**",
            "",
            "📋 **ИЗВЛЕЧЕННЫЕ ДАННЫЕ:**",
            f"• 🏠 Адреса: {', '.join(data['extracted_data']['addresses']) or 'не найдены'}",
            f"• 📅 Даты: {', '.join(data['extracted_data']['dates']) or 'не найдены'}",
            f"• 🚨 Проблемы: {', '.join(data['extracted_data']['problems']) or 'не найдены'}",
            "",
            " **РЕКОМЕНДАЦИИ:**",
            f"• ⚡ Срочность: {data['urgency_level'].upper()}",
            f"•  Категория: {category_display}",
            f"• ⏱️ Сроки решения: {recommendations['deadlines']}",
            "",
            " **НЕОБХОДИМЫЕ ДЕЙСТВИЯ:**"
        ]

        for i, action in enumerate(recommendations['actions'], 1):
            report.append(f"{i}. {action}")

        if recommendations['recommended_specialists']:
            report.extend([
                "",
                "👨‍💼 **РЕКОМЕНДУЕМЫЕ СПЕЦИАЛИСТЫ:**",
            ])
            for specialist in recommendations['recommended_specialists']:
                report.append(f"• {specialist}")

        report.extend([
            "",
            "📞 **ЭКСТРЕННЫЕ КОНТАКТЫ:**",
        ])

        for contact in recommendations['contacts']:
            report.append(f"• {contact}")

        if recommendations.get('legal_basis'):
            report.extend([
                "",
                "⚖️ **ПРАВОВАЯ БАЗА:**",
                recommendations['legal_basis']
            ])

        if data['extracted_data']['additional_info']:
            report.extend([
                "",
                "🤖 **ДОПОЛНИТЕЛЬНАЯ ИНФОРМАЦИЯ:**"
            ])
            for question, answer_info in list(data['extracted_data']['additional_info'].items())[:3]:
                report.append(f"• {question}: {answer_info['answer']}")

        return "\n".join(report)

analyzer = ImprovedComplaintAnalyzer(problems_data, specialists_data)



config.json:   0%|          | 0.00/430 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/471M [00:00<?, ?B/s]

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at microsoft/Multilingual-MiniLM-L12-H384 and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


model.safetensors:   0%|          | 0.00/471M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

Device set to use cpu


Предсказание модели и формирование совета на основании предсказания

In [17]:
def predict_complaint(text):
    try:
        inputs = tokenizer(
            text,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=512
        )

        model.eval()
        with torch.no_grad():
            outputs = model(
                input_ids=inputs['input_ids'],
                attention_mask=inputs['attention_mask']
            )

        # Получаем логиты
        category_logits = outputs['category_logits']
        emotion_logits = outputs['emotion_logits']
        urgency_logits = outputs['urgency_logits']

        # Конвертируем
        category_pred = torch.argmax(category_logits, dim=1).item()
        emotion_pred = torch.argmax(emotion_logits, dim=1).item()
        urgency_pred = torch.argmax(urgency_logits, dim=1).item()

        # Преобразуем
        try:
            category_label = binarizers['category']['classes'][category_pred]
            emotion_label = binarizers['emotion']['classes'][emotion_pred]
            urgency_label = binarizers['urgency']['classes'][urgency_pred]
        except (IndexError, AttributeError, KeyError) as e:
            print(f"Error converting labels: {e}")
            category_label = f"Категория_{category_pred}"
            emotion_label = f"Эмоция_{emotion_pred}"
            urgency_label = f"Срочность_{urgency_pred}"

        return {
            'category': category_label,
            'emotion': emotion_label,
            'urgency': urgency_label
        }
    except Exception as e:
        print(f"Prediction error: {e}")
        return {
            'category': 'не определена',
            'emotion': 'нейтральная',
            'urgency': 'средняя'
        }

def get_advice(prediction):
    """Генерация совета на основе предсаказания"""
    advice_templates = {
        'высокая': "Рекомендуем срочно обратиться в аварийную службу!",
        'критическая': "🚨 НЕМЕДЛЕННО вызывайте аварийную службу!",
        'средняя': "Обратитесь в управляющую компанию в рабочее время.",
        'низкая': "Проблема будет решена в плановом порядке.",
        'high': "Рекомендуем срочно обратиться в аварийную службу!",
        'critical': "🚨 НЕМЕДЛЕННО вызывайте аварийную службу!",
        'medium': "Обратитесь в управляющую компанию в рабочее время.",
        'low': "Проблема будет решена в плановом порядке."
    }

    return advice_templates.get(prediction['urgency'],
                               "Специалист свяжется с вами в ближайшее время.")

Оформление ответа

In [18]:
def format_specialists(rag_data):
    """Форматирует блок специалистов"""
    specialists_text = ""

    for spec_info in rag_data['specialists'][:3]:
        spec = spec_info['specialist']

        specialists_text += f"""
<b>{spec['name']}</b>
Должность: {spec['position']}
Телефон: <code>{spec['phone']}</code>
Рейтинг: {spec['rating']}/5 | Район: {spec['district']}
───────────────────
"""

    return f"👨‍💼 **РЕКОМЕНДУЕМЫЕ СПЕЦИАЛИСТЫ:**\n{specialists_text}"

In [19]:
def generate_beautiful_response(prediction, analysis_result):
    """Генерирует красивый и лаконичный ответ с рекомендациями и специалистами"""

    combined_data = analysis_result['combined_analysis']
    rag_data = analysis_result['rag_result']

    addresses = combined_data['extracted_data']['addresses']
    formatted_address = "не указан"
    if addresses:
        address_parts = []
        for addr in addresses[:1]:
            if 'улица' in str(addr):
                street = str(addr).split("value='")[1].split("'")[0] if "value='" in str(addr) else "неизвестно"
                address_parts.append(f"ул. {street}")
            if 'дом' in str(addr):
                house = str(addr).split("value='")[1].split("'")[0] if "value='" in str(addr) else "неизвестно"
                address_parts.append(f"д. {house}")
            if 'квартира' in str(addr):
                apt = str(addr).split("value='")[1].split("'")[0] if "value='" in str(addr) else "неизвестно"
                address_parts.append(f"кв. {apt}")
        if address_parts:
            formatted_address = ", ".join(address_parts)

    category = rag_data['category'].replace('_', ' ').title()

    recommendations = rag_data['recommendations']
    main_actions = recommendations['actions'][:3]

    specialists_text = ""
    for i, specialist_info in enumerate(rag_data['specialists'][:3]):
        spec = specialist_info['specialist']
        specialists_text += f"👤 {spec['name']}\n"
        specialists_text += f"   📞 {spec['phone']}\n"
        specialists_text += f"   💼 {spec['position']}\n"
        if i < 2:
            specialists_text += "   ───────────────────\n"


    response = f"""
🎯 **АНАЛИЗ ВАШЕЙ ЗАЯВКИ**

🚀 **ЧТО ДЕЛАТЬ ПРЯМО СЕЙЧАС:**
"""

    for i, action in enumerate(main_actions, 1):
        response += f"   {i}. {action}\n"

    response += f"""
📞 **КОНТАКТЫ ДЛЯ СВЯЗИ:**
   • {recommendations['contacts'][0]} - основной контакт
   • {recommendations['contacts'][1]} - дополнительный

👨‍💼 **РЕКОМЕНДУЕМЫЕ СПЕЦИАЛИСТЫ:**
{specialists_text}

⏱️ **Сроки решения:** {recommendations['deadlines']}


💡 **ВАЖНО:** {get_advice(prediction)}

💡 **Проверяем данные:**
📍 **Адрес:** {formatted_address}
🏷️ **Категория проблемы:** {category}
⚡ **Уровень срочности:** {prediction['urgency'].upper()}
😊 **Эмоциональный фон:** {prediction['emotion']}



"""

    return response

Запуск бота и формирование ответа

In [ ]:
nest_asyncio.apply()


ALLOWED_UPDATES = ['message', 'edited_message']

bot = Bot(token=TOKEN)
dp = Dispatcher()


# ==================== ТЕЛЕГРАММ БОТ ====================
@dp.message(Command("start"))
async def send_welcome(message: Message):
    await message.answer(
        "👋 Привет! Я бот для обработки жалоб ЖКХ.\n"
        "Просто опишите свою проблему, и я её классифицирую!\n\n"
        "Пример: 'В подъезде не горит свет, очень страшно'"
    )


@dp.message()
async def handle_complaint(message: Message):
    user_text = message.text

    thinking_msg = await message.answer("🔍 Анализирую вашу жалобу...")

    try:
        prediction = predict_complaint(user_text)
        analysis_result = analyzer.analyze_complaint(user_text)

        beautiful_response = generate_beautiful_response(prediction, analysis_result)

        await thinking_msg.delete()
        await message.answer(beautiful_response, parse_mode=ParseMode.MARKDOWN)

    except Exception as e:
        await thinking_msg.delete()
        error_details = f"❌ Ошибка: {str(e)}"
        await message.answer(error_details)
        print(f"Error details: {e}")

async def main():
    print("🤖 Бот запускается...")
    await bot.delete_webhook(drop_pending_updates=True)
    await dp.start_polling(bot, allowed_updates=ALLOWED_UPDATES)

asyncio.run(main())

🤖 Бот запускается...
